In [1]:
%autosave 0

In [2]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))
from os import remove


from datetime import date, timedelta
import codecs
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

from os.path import join, basename

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime


today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)


---------------------------------------------------------------

root: C:\sesa\dev\bulletin
parent: C:\sesa\dev
default_input: C:\sesa\dev\input
default_output: C:\sesa\dev\output

---------------------------------------------------------------



In [3]:
import pyminizip as pz

In [4]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [5]:
mb = Metabase()
# intervalo = "classificacao_final = 2 AND excluir_ficha = 2 AND status_notificacao in (1,2) AND ((data_notificacao >= NOW() - INTERVAL '7 DAY') or (data_liberacao >= NOW() - INTERVAL '7 DAY') or (updated_at >= NOW() - INTERVAL '7 DAY'))"
# mb.generate_notifica_query('update_notifica', where=intervalo)
# mb.list_sql_files()

limit: 100000
Cookie:'13646********-****-****-****-************27d89'

sql_files:
	0: all_notifica
	1: coisa
	2: comorbidades
	3: diario
	4: instituicao
	5: new_query
	6: old_diario
	7: query_linda
	8: update_notifica
	9: usecols_notifica

sql_results:
	0: comorbidades_len
	1: diario_len
	2: instituicao_len
	3: query_linda_len
	4: update_notifica_len


In [6]:
# update_notifica = mb.download_notificacao('update_notifica', load=False, normalize=True)
# update_notifica.save()
update_notifica = Notifica('update_notifica')
update_notifica.load()
len(update_notifica.df)

databases: ['cc_11_06_2021', 'cc_15_06_2021', 'cc_16_06_2021', 'cc_17_06_2021']


41869

In [7]:
# notifica = Notifica()
# notifica.load()
# notifica.update(update_notifica.df)
# notifica.save()
# notifica.df = notifica.df.drop_duplicates('id')
# len(notifica.df)
notifica = update_notifica

In [8]:
casos_notifica = notifica.df
casos_notifica = casos_notifica.loc[casos_notifica['data_diagnostico'].notna()].copy()
casos_notifica.loc[casos_notifica['data_liberacao'].notna(),'hash_lib'] = casos_notifica.loc[casos_notifica['data_liberacao'].notna(),'paciente'].apply(normalize_hash) + casos_notifica.loc[casos_notifica['data_liberacao'].notna(),'data_liberacao'].apply(date_hash)
casos_notifica.loc[casos_notifica['data_diagnostico'].notna(),'hash_diag'] = casos_notifica.loc[casos_notifica['data_diagnostico'].notna(),'paciente'].apply(normalize_hash) + casos_notifica.loc[casos_notifica['data_diagnostico'].notna(),'data_diagnostico'].apply(date_hash)

In [9]:
casos_notifica = pd.merge(casos_notifica.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
casos_notifica = pd.merge(casos_notifica.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})
casos_notifica = casos_notifica.loc[casos_notifica['rs'].notna()]
casos_notifica = casos_notifica.loc[casos_notifica['pais_residencia']==1]

casos_notifica = casos_notifica.loc[casos_notifica['sexo']!=3]
casos_notifica.loc[casos_notifica['sexo']==1,'sexo'] = 'M'
casos_notifica.loc[casos_notifica['sexo']==2,'sexo'] = 'F'

casos_notifica

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend
0,78463,2020-04-27,2020-09-01,2,1,2021-06-17,None,2,1,1.0,2020-09-01,3,None,08595871930,NaN,NaN,TALES AUGUSTO ZABLOSKI,M,1995-02-23,LUCELIA CADAVAL DO NASCIMENTO,1,NaN,Não Informado,NaN,NaN,1,,41,412550,83085050,Rua Giocondo Dall Stella,Quississana,41,410690,2020-08-23,2020-08-21,2020-08-23,2020-08-24,200207060478,9.0,27,1,INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA - IBMP,2020-04-13,2020-08-21,25,TALESAUGUSTOZABLOSKI25412550,TALESAUGUSTOZABLOSKI25410690,TALESAUGUSTOZABLOSKILUCELIACADAVALDONASCIMENTO,TALESAUGUSTOZABLOSKI23021995,TALESAUGUSTOZABLOSKI24082020,TALESAUGUSTOZABLOSKI21082020,2,SAO JOSE DOS PINHAIS,PR,São José dos Pinhais,CURITIBA
1,83391,2020-05-01,2021-03-17,2,1,2021-06-14,None,2,1,1.0,2021-02-14,3,None,10167063910,NaN,NaN,DEBORA CRISTINA DE LIMA,F,1996-12-03,MARGARIDA FAUSTINO DE LIMA,99,NaN,Não Informado,NaN,NaN,1,,41,411370,86057130,Rua Sebastião Roberto,Colúmbia,41,411370,2021-01-30,2021-01-28,2021-01-30,2021-01-30,211753004710,9.0,27,1,UNIMED LONDRINA,2021-01-25,2021-01-28,23,DEBORACRISTINADELIMA23411370,DEBORACRISTINADELIMA23411370,DEBORACRISTINADELIMAMARGARIDAFAUSTINODELIMA,DEBORACRISTINADELIMA03121996,DEBORACRISTINADELIMA30012021,DEBORACRISTINADELIMA28012021,17,LONDRINA,PR,Londrina,LONDRINA
2,90170,2020-04-04,NaT,1,4,2021-06-16,None,2,1,NaN,NaT,3,None,10495899984,NaN,NaN,ELIZANGELA GUERBIS DA SILVA LIMA,F,1994-11-17,None,99,5.0,,5199.0,NaN,1,,41,411370,NaN,chacara,JD NOVA ESPERANÇA,99,999999,NaT,2021-06-15,NaT,NaT,-1,12.0,167,2,,2021-06-13,2021-06-15,25,ELIZANGELAGUERBISDASILVALIMA25411370,ELIZANGELAGUERBISDASILVALIMA25999999,NaN,ELIZANGELAGUERBISDASILVALIMA17111994,NaN,ELIZANGELAGUERBISDASILVALIMA15062021,17,LONDRINA,PR,Londrina,IGNORADO/EXTERIOR/IG
3,91063,2020-03-30,2021-06-15,2,4,2021-06-15,None,2,1,1.0,2021-06-01,3,None,02691484939,NaN,NaN,DANIELA DE SOUZA MELO,F,1978-06-07,MATILDE FRANCISCA DE SOUZA,99,5.0,,NaN,NaN,1,,41,411370,NaN,GARCIA RODRIGUES PAES,0,41,411370,NaT,2020-09-02,NaT,NaT,-1,9.0,27,1,,2020-03-29,2020-09-02,41,DANIELADESOUZAMELO41411370,DANIELADESOUZAMELO41411370,DANIELADESOUZAMELOMATILDEFRANCISCADESOUZA,DANIELADESOUZAMELO07061978,NaN,DANIELADESOUZAMELO02092020,17,LONDRINA,PR,Londrina,LONDRINA
4,91117,2020-03-27,2021-01-21,2,4,2021-06-14,None,2,1,1.0,2021-01-21,3,705102374765940,53533755991,NaN,NaN,ELISETE APARECIDA ALMEIDA,F,1968-06-21,None,99,5.0,,4110.0,NaN,1,,41,411370,86035340,MANGABA,PANORAMA,99,999999,2021-01-12,2021-01-12,NaT,NaT,211703006856,9.0,181,1,UEL,2021-01-11,2021-01-12,51,ELISETEAPARECIDAALMEIDA51411370,ELISETEAPARECIDAALMEIDA51999999,NaN,ELISETEAPARECIDAALMEIDA21061968,NaN,ELISETEAPARECIDAALMEIDA12012021,17,LONDRINA,PR,Londrina,IGNORADO/EXTERIOR/IG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41864,4288380,2021-06-17,NaT,1,1,2021-06-17,None,2,1,3.0,NaT,3,702602754031540,10023380934,NaN,NaN,AMAZONAS SILVA DE SOUZA,M,1947-07-25,JUVENTINA SILVA GOMES,99,NaN,,NaN,NaN,1,,41,412170,8.432e+07,NaN,NaN,41,412170,NaT,2021-06-17,NaT,2021-06-17,-1,12.0,167,2,,2021-06-16,2021-06-17,73,AMAZONASSILVADESOUZA73412170,AMAZONASSILVADESOUZA73412170,AMAZONASSILVADESOUZAJUVENTINASILVAGOMES,AMAZONASSILVADESOUZA25071947,AMAZONASSI

In [10]:
cc = CasosConfirmados()
cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}")

databases: ['cc_11_06_2021', 'cc_15_06_2021', 'cc_16_06_2021', 'cc_17_06_2021']
Load C:\sesa\dev\bulletin\database\casos_confirmados\cc_16_06_2021.pkl
loading Casos Confirmados from pkl time elepsed:00:00:10


In [11]:
casos_confirmados = cc.df
# casos_confirmados['id_notifica'] = casos_confirmados['id_notifica'].astype(int)
# casos_confirmados['ibge_resid'] = casos_confirmados['ibge_resid'].astype(int)
# casos_confirmados['ibge_atend'] = casos_confirmados['ibge_atend'].astype(int)
# casos_confirmados['idade'] = casos_confirmados['idade'].astype(int)
# casos_confirmados['evolucao'] = casos_confirmados['evolucao'].astype(int)

# casos_confirmados['hash'] = ( casos_confirmados['nome'].apply(normalize_hash) +
#                casos_confirmados['idade'].astype(str) +
#                casos_confirmados['ibge_resid'].apply(str) )

# casos_confirmados['hash_less'] = ( casos_confirmados['nome'].apply(normalize_hash) +
#                     casos_confirmados['idade'].apply(lambda x: str(x-1)) +
#                     casos_confirmados['ibge_resid'].apply(str) )

# casos_confirmados['hash_more'] = ( casos_confirmados['nome'].apply(normalize_hash) +
#                     casos_confirmados['idade'].apply(lambda x: str(x+1)) +
#                     casos_confirmados['ibge_resid'].apply(str) )

# casos_confirmados['hash_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
#                     casos_confirmados['idade'].astype(str) +
#                     casos_confirmados['ibge_atend'].apply(str) )

# casos_confirmados['hash_less_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
#                     casos_confirmados['idade'].apply(lambda x: str(x-1)) +
#                     casos_confirmados['ibge_atend'].apply(str) )

# casos_confirmados['hash_more_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
#                     casos_confirmados['idade'].apply(lambda x: str(x+1)) +
#                     casos_confirmados['ibge_atend'].apply(str) )

# casos_confirmados['hash_diag'] = ( casos_confirmados['nome'].apply(normalize_hash) +
#                     casos_confirmados['dt_diag'].apply(date_hash) )
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,is2,com,evo,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaT,1,2020-04-03,NaT,NaN,2020-03-12,2020-04-03 00:00:00,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07,1,NaT,NaT,2020-03-07 00:00:00,2020-03-12,NaN,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08,1,NaT,NaT,2020-03-08 00:00:00,2020-03-12,NaN,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11,1,NaT,NaT,2020-03-11 00:00:00,2020-03-12,NaN,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE16032020
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15,1,NaT,NaT,2020-03-15 00:00:00,2020-03-12,NaN,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN18032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162961,1252007,-1,PR,412440,412440,ZONIRA FERNANDES BUENO MINETTO,F,54,167,2021-06-15,2021-06-16,2021-06-10,3,NaT,NaT,2021-06-10 00:00:00,2021-06-16,NaN,ZONIRAFERNANDESBUENOMINETTO54412440,ZONIRAFERNANDESBUENOMINETTO53412440,ZONIRAFERNANDESBUENOMINETTO55412440,ZONIRAFERNANDESBUENOMINETTO54412440,ZONIRAFERNANDESBUENOMINETTO53412440,ZONIRAFERNANDESBUENOMINETTO55412440,ZONIRAFERNANDESBUENOMINETTO15062021
1162962,1249899,-1,PR,410830,410830,ZULEIKA TEREZINHA TORRES PEREIRA,F,37,181,2021-06-14,2021-06-16,2021-06-11,3,NaT,NaT,2021-06-11 00:00:00,2021-06-16,NaN,ZULEIKATEREZINHATORRESPEREIRA37410830,ZULEIKATEREZINHATORRESPEREIRA36410830,ZULEIKATEREZINHATORRESPEREIRA38410830,ZULEIKATEREZINHATORRESPEREIRA37410830,ZULEIKATEREZINHATORRESPEREIRA36410830,ZULEIKATEREZINHATORRESPEREIRA38410830,ZULEIKATEREZINHATORRESPEREIRA14062021
1162963,1254288,-1,PR,410430,410430,ZULMIRA BERANISE DE OLIVEIRA,F,50,1,2021-04-20,2021-06-16,2021-04-14,1,2021-05-21,2021-06-16,2021-04-14 00:00:00,2021-06-16,2021-05-21 00:00:00,ZULMIRABERANISEDEOLIVEIRA50410430,ZULMIRABERANISEDEOLIVEIRA49410430,ZULMIRABERANISEDEOLIVEIRA51410430,ZULMIRABERANISEDEOLIVEIRA50410430,ZULMIRABERANISEDEOLIVEIRA49410430,ZULMIRABERANISEDEOLIVEIRA51410430,ZULMIRABERANISEDEOLIVEIRA20042021
1162964,1249900,-1,PR,411722,411722,ZUNILDA SARACHO SANTIN,F,49,27,2021-06-14,2021-06-16,2021-06-04,3,NaT,NaT,2021-06-04 00:00:00,2021-06-16,NaN,ZUNILDASARACHOSANTIN49411722,ZUNILDASARACHOSANTIN48411722,ZUNILDASARACHOSANTIN50411722,ZUNILDASARACHOSANTIN49411722,ZUNILDASARACHOSANTIN48411722,ZUNILDASARACHOSANTIN50411722,ZUNILDASARACHOSANTIN14062021


In [12]:
# casos_confirmados['is'] = casos_confirmados['is'].apply(lambda x: str(x)[:10] if not str(x).lower() in ['none','nat','nan'] else 'none')
# casos_confirmados.loc[casos_confirmados['is'].str.contains('/'),'is2'] = casos_confirmados.loc[casos_confirmados['is'].str.contains('/'),'is'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
# casos_confirmados.loc[casos_confirmados['is'].str.contains('-'),'is2'] = casos_confirmados.loc[casos_confirmados['is'].str.contains('-'),'is'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

# casos_confirmados['comunicacao'] = casos_confirmados['comunicacao'].apply(str)
# casos_confirmados.loc[casos_confirmados['comunicacao'].str.contains('/'),'com'] = casos_confirmados.loc[casos_confirmados['comunicacao'].str.contains('/'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
# casos_confirmados.loc[casos_confirmados['comunicacao'].str.contains('-'),'com'] = casos_confirmados.loc[casos_confirmados['comunicacao'].str.contains('-'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

# casos_confirmados['data_evolucao'] = casos_confirmados['data_evolucao'].apply(str)
# casos_confirmados.loc[casos_confirmados['data_evolucao'].str.contains('/'),'evo'] = casos_confirmados.loc[casos_confirmados['data_evolucao'].str.contains('/'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='coerce'))
# casos_confirmados.loc[casos_confirmados['data_evolucao'].str.contains('-'),'evo'] = casos_confirmados.loc[casos_confirmados['data_evolucao'].str.contains('-'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='coerce'))

# casos_confirmados['id_notifica'] = casos_confirmados['id_notifica'].apply(int)
# casos_confirmados['evolucao'] = casos_confirmados['evolucao'].apply(int)
# casos_confirmados['is'] = casos_confirmados['is2']
# casos_confirmados['comunicacao'] = casos_confirmados['com']
# casos_confirmados['data_evolucao'] = casos_confirmados['evo'].apply(lambda x: pd.to_datetime(x))

casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

,identificacao
evolucao,
1,818586
2,28847
3,315533


In [13]:
casos_confirmados = pd.merge(casos_confirmados.rename(columns={'ibge_resid':'ibge'}),municipios,how='left',on='ibge').rename(columns={'ibge':'ibge_resid'})

In [14]:
# duplicados = casos_confirmados.loc[casos_confirmados.duplicated('hash',keep='first')]
# casos_confirmados = casos_confirmados.drop(index=duplicados.index)

# duplicados_hash_diag = casos_confirmados.loc[casos_confirmados.duplicated('hash_diag',keep='first')]
# duplicados_hash_diag = duplicados_hash_diag.loc[duplicados_hash_diag['id_notifica']!=-1]
# casos_confirmados = casos_confirmados.drop(index=duplicados_hash_diag.index)

# duplicados = duplicados.append(duplicados_hash_diag)

# duplicados_hash_diag = casos_confirmados.loc[casos_confirmados.duplicated('hash_diag',keep='first')]
# duplicados_hash_diag = duplicados_hash_diag.iloc[:250-len(duplicados)]
# casos_confirmados = casos_confirmados.drop(index=duplicados_hash_diag.index)

# duplicados = duplicados.append(duplicados_hash_diag)

# duplicados = pd.merge(duplicados.rename(columns={'ibge_resid':'ibge'}),municipios,on='ibge',how='left')

# duplicados

In [15]:
# duplicados.to_pickle(join(default_output,'exclusoes.pkl'))
# duplicados.to_excel(join(default_output,'exclusoes.xlsx'))
# duplicados[['identificacao','evolucao','rs']].groupby(['rs','evolucao']).count()

In [16]:
# notifica_df = notifica.df
# notifica_df['dt_diag'] = notifica_df['data_diagnostico']

In [17]:
# arrumar_datas = casos_confirmados.loc[casos_confirmados['dt_diag'].isna()].copy()
# del arrumar_datas['dt_diag']
# arrumar_datas = pd.merge(arrumar_datas,notifica_df[['id','dt_diag']].rename(columns={'id':'id_notifica'}), on='id_notifica', how='left')
# arrumar_datas

In [18]:
# casos_confirmados = casos_confirmados.set_index('identificacao')
# arrumar_datas = arrumar_datas.set_index('identificacao')
# casos_confirmados.update(arrumar_datas)
# casos_confirmados.reset_index(inplace=True)
casos_notifica

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend
0,78463,2020-04-27,2020-09-01,2,1,2021-06-17,None,2,1,1.0,2020-09-01,3,None,08595871930,NaN,NaN,TALES AUGUSTO ZABLOSKI,M,1995-02-23,LUCELIA CADAVAL DO NASCIMENTO,1,NaN,Não Informado,NaN,NaN,1,,41,412550,83085050,Rua Giocondo Dall Stella,Quississana,41,410690,2020-08-23,2020-08-21,2020-08-23,2020-08-24,200207060478,9.0,27,1,INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA - IBMP,2020-04-13,2020-08-21,25,TALESAUGUSTOZABLOSKI25412550,TALESAUGUSTOZABLOSKI25410690,TALESAUGUSTOZABLOSKILUCELIACADAVALDONASCIMENTO,TALESAUGUSTOZABLOSKI23021995,TALESAUGUSTOZABLOSKI24082020,TALESAUGUSTOZABLOSKI21082020,2,SAO JOSE DOS PINHAIS,PR,São José dos Pinhais,CURITIBA
1,83391,2020-05-01,2021-03-17,2,1,2021-06-14,None,2,1,1.0,2021-02-14,3,None,10167063910,NaN,NaN,DEBORA CRISTINA DE LIMA,F,1996-12-03,MARGARIDA FAUSTINO DE LIMA,99,NaN,Não Informado,NaN,NaN,1,,41,411370,86057130,Rua Sebastião Roberto,Colúmbia,41,411370,2021-01-30,2021-01-28,2021-01-30,2021-01-30,211753004710,9.0,27,1,UNIMED LONDRINA,2021-01-25,2021-01-28,23,DEBORACRISTINADELIMA23411370,DEBORACRISTINADELIMA23411370,DEBORACRISTINADELIMAMARGARIDAFAUSTINODELIMA,DEBORACRISTINADELIMA03121996,DEBORACRISTINADELIMA30012021,DEBORACRISTINADELIMA28012021,17,LONDRINA,PR,Londrina,LONDRINA
2,90170,2020-04-04,NaT,1,4,2021-06-16,None,2,1,NaN,NaT,3,None,10495899984,NaN,NaN,ELIZANGELA GUERBIS DA SILVA LIMA,F,1994-11-17,None,99,5.0,,5199.0,NaN,1,,41,411370,NaN,chacara,JD NOVA ESPERANÇA,99,999999,NaT,2021-06-15,NaT,NaT,-1,12.0,167,2,,2021-06-13,2021-06-15,25,ELIZANGELAGUERBISDASILVALIMA25411370,ELIZANGELAGUERBISDASILVALIMA25999999,NaN,ELIZANGELAGUERBISDASILVALIMA17111994,NaN,ELIZANGELAGUERBISDASILVALIMA15062021,17,LONDRINA,PR,Londrina,IGNORADO/EXTERIOR/IG
3,91063,2020-03-30,2021-06-15,2,4,2021-06-15,None,2,1,1.0,2021-06-01,3,None,02691484939,NaN,NaN,DANIELA DE SOUZA MELO,F,1978-06-07,MATILDE FRANCISCA DE SOUZA,99,5.0,,NaN,NaN,1,,41,411370,NaN,GARCIA RODRIGUES PAES,0,41,411370,NaT,2020-09-02,NaT,NaT,-1,9.0,27,1,,2020-03-29,2020-09-02,41,DANIELADESOUZAMELO41411370,DANIELADESOUZAMELO41411370,DANIELADESOUZAMELOMATILDEFRANCISCADESOUZA,DANIELADESOUZAMELO07061978,NaN,DANIELADESOUZAMELO02092020,17,LONDRINA,PR,Londrina,LONDRINA
4,91117,2020-03-27,2021-01-21,2,4,2021-06-14,None,2,1,1.0,2021-01-21,3,705102374765940,53533755991,NaN,NaN,ELISETE APARECIDA ALMEIDA,F,1968-06-21,None,99,5.0,,4110.0,NaN,1,,41,411370,86035340,MANGABA,PANORAMA,99,999999,2021-01-12,2021-01-12,NaT,NaT,211703006856,9.0,181,1,UEL,2021-01-11,2021-01-12,51,ELISETEAPARECIDAALMEIDA51411370,ELISETEAPARECIDAALMEIDA51999999,NaN,ELISETEAPARECIDAALMEIDA21061968,NaN,ELISETEAPARECIDAALMEIDA12012021,17,LONDRINA,PR,Londrina,IGNORADO/EXTERIOR/IG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41864,4288380,2021-06-17,NaT,1,1,2021-06-17,None,2,1,3.0,NaT,3,702602754031540,10023380934,NaN,NaN,AMAZONAS SILVA DE SOUZA,M,1947-07-25,JUVENTINA SILVA GOMES,99,NaN,,NaN,NaN,1,,41,412170,8.432e+07,NaN,NaN,41,412170,NaT,2021-06-17,NaT,2021-06-17,-1,12.0,167,2,,2021-06-16,2021-06-17,73,AMAZONASSILVADESOUZA73412170,AMAZONASSILVADESOUZA73412170,AMAZONASSILVADESOUZAJUVENTINASILVAGOMES,AMAZONASSILVADESOUZA25071947,AMAZONASSI

In [19]:
# novos_casos = casos_notifica.loc[~(
#     (casos_notifica['id'].isin(casos_confirmados['id_notifica'])) |
#     (casos_notifica['hash'].isin(casos_confirmados['hash'])) |
#     (casos_notifica['hash'].isin(casos_confirmados['hash_less'])) |
#     (casos_notifica['hash'].isin(casos_confirmados['hash_more'])) |
#     (casos_notifica['hash_atend'].isin(casos_confirmados['hash_atend'])) |
#     (casos_notifica['hash_atend'].isin(casos_confirmados['hash_less_atend'])) |
#     (casos_notifica['hash_atend'].isin(casos_confirmados['hash_more_atend'])) |
#     (casos_notifica['hash_diag'].isin(casos_confirmados['hash_diag'])) |
#     (casos_notifica['hash_lib'].isin(casos_confirmados['hash_diag']))
# )].copy()

# novos_casos['data_com'] = pd.to_datetime(date.today())
# novos_casos = novos_casos.sort_values('paciente')

# novos_casos = novos_casos.sample(n=7666)

novos_casos_excel = pd.read_excel(join(default_output,'novos_casos.xlsx'))
novos_casos = casos_notifica.loc[casos_notifica['id'].isin(novos_casos_excel['id'])].copy()
novos_casos['data_com'] = pd.to_datetime(date.today())
novos_casos

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend,data_com
20,133070,2020-05-21,NaT,1,1,2021-06-17,None,2,1,3.0,NaT,3,700202490229624,09052404992,NaN,NaN,VINICIUS VIZOTTO ALVES,M,1998-05-26,CRISTIANE BARBOSA VIZOTTO ALVES,99,5.0,COMERCIAL IVAIPORÃ - MATERIAL DE CONSTRUÇÃO,NaN,NaN,1,,41,411840,NaN,RUA OZIRES RAMIRO DE ASSIS,JD ANTIGO AEROPORTO,41,411840,NaT,NaT,NaT,NaT,-1,12.0,167,2,,2021-06-10,2020-05-21,21,VINICIUSVIZOTTOALVES21411840,VINICIUSVIZOTTOALVES21411840,VINICIUSVIZOTTOALVESCRISTIANEBARBOSAVIZOTTOALVES,VINICIUSVIZOTTOALVES26051998,NaN,VINICIUSVIZOTTOALVES21052020,14,PARANAVAI,PR,Paranavaí,PARANAVAI,2021-06-17
22,139160,2020-04-25,NaT,1,2,2021-06-17,None,2,1,NaN,2021-06-17,3,None,03690564930,NaN,NaN,VALDOMIRO MOREIRA DE ANDRADE FILHO,M,1982-09-30,VERA LUCIA PEREIRA DE ANDRADE,1,NaN,Não Informado,NaN,NaN,1,,41,412080,83420000,TERTULIANO DE LARA,SANTA LUZIA,41,410690,2021-06-02,2021-05-30,NaT,NaT,210206207390,9.0,27,1,IBMP,2021-05-24,2021-05-30,37,VALDOMIROMOREIRADEANDRADEFILHO37412080,VALDOMIROMOREIRADEANDRADEFILHO37410690,VALDOMIROMOREIRADEANDRADEFILHOVERALUCIAPEREIRA...,VALDOMIROMOREIRADEANDRADEFILHO30091982,NaN,VALDOMIROMOREIRADEANDRADEFILHO30052021,2,QUATRO BARRAS,PR,Quatro Barras,CURITIBA,2021-06-17
26,141140,2020-04-06,NaT,1,2,2021-06-17,None,2,1,NaN,NaT,3,702308131539316,08917011926,NaN,NaN,ANDERSON GOIS DE FREITAS,M,1993-09-10,NEUVAIR OLIVEIRA GOIS,1,NaN,Não Informado,NaN,NaN,1,,41,410690,82115270,MARIA JOANA FARIAS,PILARZINHO,41,410690,NaT,2021-06-17,NaT,NaT,-1,12.0,167,2,,2021-06-15,2021-06-17,26,ANDERSONGOISDEFREITAS26410690,ANDERSONGOISDEFREITAS26410690,ANDERSONGOISDEFREITASNEUVAIROLIVEIRAGOIS,ANDERSONGOISDEFREITAS10091993,NaN,ANDERSONGOISDEFREITAS17062021,2,CURITIBA,PR,Curitiba,CURITIBA,2021-06-17
39,150762,2020-04-26,2021-06-16,2,2,2021-06-16,None,2,1,2.0,2021-06-14,3,706004884352940,32137265591,NaN,NaN,IRENE GONCALVES DE OLIVEIRA,F,1963-01-04,ALMERINDA LUIZA DE OLIVEIRA,1,NaN,Não Informado,NaN,NaN,1,,41,410690,81935480,GENERAL ADALBERTO MASSA,GANCHINHO,41,410690,2021-05-31,2021-05-31,2021-06-01,2021-06-02,210206209084,9.0,27,1,INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA - IBMP,2021-05-28,2021-05-31,57,IRENEGONCALVESDEOLIVEIRA57410690,IRENEGONCALVESDEOLIVEIRA57410690,IRENEGONCALVESDEOLIVEIRAALMERINDALUIZADEOLIVEIRA,IRENEGONCALVESDEOLIVEIRA04011963,IRENEGONCALVESDEOLIVEIRA02062021,IRENEGONCALVESDEOLIVEIRA31052021,2,CURITIBA,PR,Curitiba,CURITIBA,2021-06-17
122,398990,2021-06-16,NaT,1,12,2021-06-16,None,2,1,3.0,NaT,3,700806402437082,03289679985,NaN,NaN,SERGIO RIBEIRO DE LIMA,M,1981-04-16,DORVALINA RODRIGUES DE LIMA,1,NaN,administrador,NaN,NaN,1,,41,411320,NaN,NaN,NaN,41,410690,2020-07-13,2021-06-16,2020-07-13,2020-07-13,-1,12.0,167,2,,2021-06-14,2021-06-16,40,SERGIORIBEIRODELIMA40411320,SERGIORIBEIRODELIMA40410690,SERGIORIBEIRODELIMADORVALINARODRIGUESDELIMA,SERGIORIBEIRODELIMA16041981,SERGIORIBEIRODELIMA13072020,SERGIORIBEIRODELIMA16062021,2,LAPA,PR,Lapa,CURITIBA,2021-06-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41848,4288311,2021-06-17,NaT,1,1,2021-06-17,None,2,1,3.0,NaT,3,700508102255250,01712111973,NaN,NaN,MARIA GABRIELA DA SILVA,F,1936-06-24,MARIA MADALENA MORAIS,99,NaN,Não Informa

In [20]:
# casos_retroativos = pd.read_pickle('casos_retroativos.pkl')
# casos_retroativos.sample(n=1000)

In [21]:
# novos_casos[['id','paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'exame','data_diagnostico','data_com','data_1o_sintomas','hash']].to_excel(join(default_output,'novos_casos.xlsx'))

In [22]:
# obitos_notifica = casos_notifica.loc[casos_notifica['evolucao']==2]
# obitos_casos = casos_confirmados.loc[casos_confirmados['evolucao']==2]

# novos_obitos = obitos_notifica.loc[~(
#     (obitos_notifica['id'].isin(obitos_casos['id_notifica'])) |
#     (obitos_notifica['hash'].isin(obitos_casos['hash'])) |
#     (obitos_notifica['hash'].isin(obitos_casos['hash_less'])) |
#     (obitos_notifica['hash'].isin(obitos_casos['hash_more'])) |
#     (obitos_notifica['hash_atend'].isin(obitos_casos['hash_atend'])) |
#     (obitos_notifica['hash_atend'].isin(obitos_casos['hash_less_atend'])) |
#     (obitos_notifica['hash_atend'].isin(obitos_casos['hash_more_atend'])) |
#     (obitos_notifica['hash_diag'].isin(obitos_casos['hash_diag']))
# )]

# novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]
# novos_obitos = novos_obitos.loc[novos_obitos['hash'].isin(casos_confirmados['hash'].tolist() + novos_casos['hash'].tolist())]
# novos_obitos = novos_obitos.sort_values(['paciente'])

# novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]

# novos_obitos = novos_obitos.sample(n=330)

# novos_obitos = novos_obitos.sample(n=301)
# novos_obitos1 = novos_obitos.iloc[-201:]
# novos_obitos2 = novos_obitos.iloc[:100]
# novos_obitos = novos_obitos1.append(novos_obitos2)
# novos_obitos = novos_obitos.drop_duplicates('hash', keep=False)

novos_obitos_excel = pd.read_excel(join(default_output,'novos_obitos.xlsx'))
novos_obitos = casos_notifica.loc[casos_notifica['id'].isin(novos_obitos_excel['id'])]
novos_obitos

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend
39,150762,2020-04-26,2021-06-16,2,2,2021-06-16,None,2,1,2.0,2021-06-14,3,706004884352940,32137265591,NaN,NaN,IRENE GONCALVES DE OLIVEIRA,F,1963-01-04,ALMERINDA LUIZA DE OLIVEIRA,1,NaN,Não Informado,NaN,NaN,1,,41,410690,81935480,GENERAL ADALBERTO MASSA,GANCHINHO,41,410690,2021-05-31,2021-05-31,2021-06-01,2021-06-02,210206209084,9.0,27,1,INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA - IBMP,2021-05-28,2021-05-31,57,IRENEGONCALVESDEOLIVEIRA57410690,IRENEGONCALVESDEOLIVEIRA57410690,IRENEGONCALVESDEOLIVEIRAALMERINDALUIZADEOLIVEIRA,IRENEGONCALVESDEOLIVEIRA04011963,IRENEGONCALVESDEOLIVEIRA02062021,IRENEGONCALVESDEOLIVEIRA31052021,2,CURITIBA,PR,Curitiba,CURITIBA
317,800119,2020-08-28,2021-06-16,2,2,2021-06-16,None,2,1,2.0,2021-06-14,3,705000607730353,74690671915,NaN,NaN,NAIR JUSTINA DA SILVA DE OLIVEIRA,F,1956-06-04,MARIA APARECIDA DA SILVA,3,NaN,Não Informado,NaN,NaN,1,,41,410690,82220415,DIACONO SILVERIO DA SILVA NEGRAO,CACHOEIRA,41,410690,2021-06-01,2021-06-01,2021-06-02,2021-06-03,210206211449,9.0,27,1,INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA - IBMP,2021-05-24,2021-06-01,64,NAIRJUSTINADASILVADEOLIVEIRA64410690,NAIRJUSTINADASILVADEOLIVEIRA64410690,NAIRJUSTINADASILVADEOLIVEIRAMARIAAPARECIDADASILVA,NAIRJUSTINADASILVADEOLIVEIRA04061956,NAIRJUSTINADASILVADEOLIVEIRA03062021,NAIRJUSTINADASILVADEOLIVEIRA01062021,2,CURITIBA,PR,Curitiba,CURITIBA
1003,1766678,2020-12-18,2021-03-26,2,1,2021-06-16,30702730,2,1,2.0,2020-12-31,3,703007879374476,25759884972,NaN,NaN,FILEMON BAEZ LESCANO,M,1942-07-04,MARIA DEL PILAR,3,NaN,,NaN,NaN,1,,41,410830,85851210,Avenida Juscelino Kubitschek,Centro,41,410830,NaT,2020-12-17,NaT,NaT,-1,9.0,27,1,,2020-12-15,2020-12-17,78,FILEMONBAEZLESCANO78410830,FILEMONBAEZLESCANO78410830,FILEMONBAEZLESCANOMARIADELPILAR,FILEMONBAEZLESCANO04071942,NaN,FILEMONBAEZLESCANO17122020,9,FOZ DO IGUACU,PR,Foz do Iguaçu,FOZ DO IGUACU
2172,2271327,2021-02-03,2021-06-16,2,1,2021-06-16,29336102,2,1,2.0,2020-10-05,3,700506182762055,02378615930,NaN,NaN,HELENA BARBIERI MICHELON,F,1935-03-25,MARIA BARBIERI,99,5.0,Não Informado,NaN,NaN,1,FOZ DO IGUAÇU,41,410830,85854480,FLOR DE PALHA,VILA ADRIANA,41,410830,NaT,2020-10-01,NaT,NaT,-1,12.0,182,6,,NaT,2020-10-01,85,HELENABARBIERIMICHELON85410830,HELENABARBIERIMICHELON85410830,HELENABARBIERIMICHELONMARIABARBIERI,HELENABARBIERIMICHELON25031935,NaN,HELENABARBIERIMICHELON01102020,9,FOZ DO IGUACU,PR,Foz do Iguaçu,FOZ DO IGUACU
2325,2305255,2021-02-06,2021-03-20,2,20,2021-06-16,30754510,2,1,2.0,2021-02-10,3,704003329425463,69312036815,NaN,NaN,JOAO DIONISIO GOMES,M,1949-05-20,ARISTINA DIONISIO GOMES,99,5.0,TRABALHADOR RURAl,NaN,NaN,1,,41,410830,85870730,Rua Juan Vicente Chaves,Cidade Nova II,41,410830,2021-02-06,2021-02-06,NaT,NaT,-1,9.0,27,1,,NaT,2021-02-06,71,JOAODIONISIOGOMES71410830,JOAODIONISIOGOMES71410830,JOAODIONISIOGOMESARISTINADIONISIOGOMES,JOAODIONISIOGOMES20051949,NaN,JOAODIONISIOGOMES06022021,9,FOZ DO IGUACU,PR,Foz do Iguaçu,FOZ DO IGUACU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38240,4269053,2021-06-16,2021-06-16,2,1,2021-06-16,None,2,1,2.0,2021-06-14,3,700204449930722,53613821915,NaN,NaN,ADAIR MARTINS NEVES,M,1964-03-22,ROSARIA

In [23]:
# novos_obitos[['id','paciente','sexo','idade','mun_resid', 'rs', 'data_cura_obito','hash']].to_excel(join(default_output,'novos_obitos.xlsx'))

In [24]:
# novos_obitos.to_excel('casos_retroativos.xlsx')
# novos_casos.to_excel('obitos_retroativos.xlsx')
# novos_obitos.to_pickle('obitos_retroativos.pkl')
# novos_casos.to_pickle('casos_retroativos.pkl')

In [25]:
date_parser = lambda x: x

In [26]:
casos_confirmadosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]

obitos_confirmados =  casos_confirmados.loc[casos_confirmados['evolucao']==2]

obitos_confirmadosPR = obitos_confirmados.loc[obitos_confirmados['rs']!=99]

print(f"Total de casos: {len(casos_confirmados)} + {len(novos_casos)}")
print(f"Total de obitos: {len(obitos_confirmados)} + {len(novos_obitos)}\n\n")

novos_casosPR = novos_casos.loc[novos_casos['rs']!=99].copy()
print(f"Total de casos PR: {len(casos_confirmadosPR)} + {len(novos_casosPR)}")

novos_obitosPR = novos_obitos.loc[novos_obitos['rs']!=99].copy()
print(f"Total de obitos PR: {len(obitos_confirmadosPR)} + {len(novos_obitosPR)}")

novos_casosFora = novos_casos.loc[novos_casos['rs']==99].copy()
print(f"Total de casos Fora: {len(casos_confirmados) - len(casos_confirmadosPR)} + {len(novos_casosFora)}")

novos_obitosFora = novos_obitos.loc[novos_obitos['rs']==99].copy()
print(f"Total de obitos Fora: {len(obitos_confirmados) - len(obitos_confirmadosPR)} + {len(novos_obitosFora)}")


novos_obitosPR_group = novos_obitosPR.groupby(by='municipio')

data_retroativos = ontem - timedelta(14)

retroativos = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) <= data_retroativos)].sort_values(by='data_diagnostico')
last2weeks = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) > data_retroativos) & (novos_casosPR['data_diagnostico'].apply(date_parser) <= anteontem)].sort_values(by='data_diagnostico')
hoje = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) > anteontem)].sort_values(by='data_diagnostico')

obitos_retroativos = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) <= data_retroativos)].sort_values(by='data_cura_obito')
obitos_last2weeks = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > data_retroativos) & (novos_obitosPR['data_cura_obito'].apply(date_parser) <= anteontem)].sort_values(by='data_cura_obito')
obitos_hoje = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > anteontem)].sort_values(by='data_cura_obito')

relatorio_file = join(default_output,f"relatorio_{(today.strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt")
with codecs.open(relatorio_file,"w","utf-8-sig") as relatorio:
    relatorio.write(f"{today.strftime('%d/%m/%Y')}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))

    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"divulgados no PR.\n")

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))

    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))

    if len(novos_obitosFora) > 0:
        relatorio.write('\n')
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n".replace(',','.'))

    relatorio.write('\n')
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))

    for _, row in novos_obitos.iterrows():
       relatorio.write(f"{row['sexo']}\t{int(row['idade'])}\t{row['municipio'] if row['rs']!=99 else (row['municipio']+'/'+row['uf'])}\t{str(int(row['rs'])).zfill(2) if row['rs']!=99 else '#N/D'}\t{row['data_cura_obito'].day}/{static.meses[row['data_cura_obito'].month-1]}/{row['data_cura_obito'].year}\n")
    relatorio.write('\n')

    if True:

        #casos
        relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(retroativos):,} casos retroativos confirmados no período de {retroativos.iloc[0]['data_diagnostico'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_diagnostico'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(last2weeks):,} novos casos confirmados no período de {last2weeks.iloc[0]['data_diagnostico'].strftime('%d/%m/%Y')} à {last2weeks.iloc[-1]['data_diagnostico'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(hoje):,} novos casos confirmados hoje.\n\n".replace(',','.'))


        novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_diagnostico'].month, axis=1)
        novos_casosPR['year'] = novos_casosPR.apply(lambda x: x['data_diagnostico'].year, axis=1)
        relatorio.write('Novos casos por meses:\n')
        for group, value in novos_casosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')
        
        relatorio.write('Novos casos por dia:\n')
        for group, value in novos_casosPR.groupby(by='data_diagnostico'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}\t{len(value)}\n")
        relatorio.write('\n')

        #obitos
        relatorio.write(f"{len(novos_obitosPR):,} novos obitos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(obitos_retroativos):,} obitos retroativos ocorridos no período de {obitos_retroativos.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_retroativos.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_last2weeks):,} novos obitos ocorridos no período de {obitos_last2weeks.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_last2weeks.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_hoje):,} novos obitos ocorridos hoje.\n\n".replace(',','.'))

        relatorio.write('Novos obitos por meses:\n')
        novos_obitosPR['month'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].month, axis=1)
        novos_obitosPR['year'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].year, axis=1)
        for group, value in novos_obitosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        relatorio.write('Novos obitos por dia:\n')
        for group, value in novos_obitosPR.groupby(by='data_cura_obito'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}\t{len(value)}\n")
        relatorio.write('\n')

        #-----RELATÓRIO DA COMUNICAÇÃO--------------
        obitos_list = []
        munic = []
        for municipio, obitos in novos_obitosPR_group:
            obito = len(obitos)
            obitos_list.append(obito)
            munic.append(municipio)             

        dicionario = (dict(zip(list(munic),list(obitos_list))))
        #print(dicionario)
        dicionario = sorted(dicionario.items(),key=lambda x: x[1], reverse = True)
        #print(dicionario)


        relatorio.write(f"\nOs pacientes que foram a óbito residiam em: ")
        for municip, obit in dict(dicionario).items():
            if obit != 1:
                relatorio.write(f"{municip} ({obit})")
                relatorio.write(f", ")
        relatorio.write(f".\n")
        relatorio.write(f"A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios:")
        for municip, obit in dict(dicionario).items():
            if obit == 1:
                relatorio.write(f"{municip}")
                relatorio.write(f", ")


Total de casos: 1162966 + 7666
Total de obitos: 28847 + 330


Total de casos PR: 1156632 + 7649
Total de obitos PR: 28683 + 330
Total de casos Fora: 6334 + 17
Total de obitos Fora: 164 + 0


In [27]:
with codecs.open(relatorio_file,"r","utf-8-sig") as relatorio:
    print(relatorio.read())

17/06/2021
7.649 novos casos residentes e 17 não residentes divulgados no PR.
1.164.281 casos confirmados residentes do PR.
1.170.632 total geral.

330 Óbitos residentes do PR:
1 Agudos do Sul
3 Almirante Tamandaré
2 Alto Piquiri
1 Ampére
9 Apucarana
2 Arapongas
1 Arapoti
1 Araruna
8 Araucária
1 Assaí
1 Assis Chateaubriand
2 Astorga
1 Atalaia
1 Balsa Nova
1 Bituruna
1 Bom Sucesso
1 Cafelândia
1 Cambará
2 Cambira
4 Cambé
3 Campina Grande do Sul
1 Campo Bonito
4 Campo Largo
3 Campo Magro
5 Campo Mourão
1 Campo do Tenente
1 Capanema
3 Carambeí
5 Cascavel
2 Castro
1 Centenário do Sul
1 Cerro Azul
4 Cianorte
6 Cidade Gaúcha
10 Colombo
2 Contenda
3 Cornélio Procópio
1 Coronel Vivida
1 Cruzeiro do Oeste
57 Curitiba
1 Cândido de Abreu
1 Faxinal
11 Fazenda Rio Grande
13 Foz do Iguaçu
4 Francisco Beltrão
1 Grandes Rios
3 Guarapuava
1 Guaraqueçaba
1 Ibaiti
1 Imbaú
2 Imbituva
1 Inajá
1 Inácio Martins
4 Irati
1 Itaperuçu
1 Jaboti
1 Jaguariaíva
1 Jandaia do Sul
1 Japurá
1 Jardim Alegre
3 Joaquim Táv

In [ ]:
while True:
    pass

In [29]:
# cc = CasosConfirmados()
# cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}")
casos_confirmados = cc.df
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

,identificacao
evolucao,
1,818586
2,28847
3,315533


In [30]:
novos_casos = novos_casos.rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'exame': 'laboratorio',
    'data_diagnostico':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao',
    'uf': 'uf_resid'
})

novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2
novos_casos.loc[~novos_casos['evolucao'].isin([1,2,3]),'evolucao'] = 3
novos_casos.loc[novos_casos['evolucao']==2,'evolucao'] = 3

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(str) +
                  novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_atend'] = (novos_casos['nome'].apply(normalize_hash) +
                    novos_casos['idade'].astype(str) +
                    novos_casos['ibge_atend'].apply(str) )

novos_casos['hash_less_atend'] = (novos_casos['nome'].apply(normalize_hash) +
                    novos_casos['idade'].apply(lambda x: str(x-1)) +
                    novos_casos['ibge_atend'].apply(str) )

novos_casos['hash_more_atend'] = (novos_casos['nome'].apply(normalize_hash) +
                    novos_casos['idade'].apply(lambda x: str(x+1)) +
                    novos_casos['ibge_atend'].apply(str) )

novos_casos['identificacao'] = list(range(casos_confirmados.iloc[-1,0] + 1,casos_confirmados.iloc[-1,0] + 1 + len(novos_casos)))
casos_confirmados = casos_confirmados.append(novos_casos)
print(len(casos_confirmados))
# casos_confirmados.groupby('evolucao')[['sexo']].count()
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

1170632


,identificacao
evolucao,
1.0,821495
2.0,28847
3.0,320290


In [31]:
novos_obitos = novos_obitos.rename(columns={
    'id': 'id_notifica',
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['hash'] = (novos_obitos['nome'].apply(normalize_hash) +
                          novos_obitos['idade'].astype(str) +
                              novos_obitos['ibge_residencia'].astype(str) )

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today())
novos_obitos['evolucao'] = 2

novos_obitos = novos_obitos[['hash','id_notifica','evolucao','data_evolucao','data_com_evolucao']]
novos_obitos = novos_obitos.set_index('hash')

casos_confirmados = casos_confirmados.set_index('hash')
casos_confirmados.update(novos_obitos)
casos_confirmados.reset_index(inplace=True)

# print(len(casos_confirmados))
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

,identificacao
evolucao,
1.0,821443
2.0,29177
3.0,320012


In [32]:
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

,identificacao
evolucao,
1.0,821443
2.0,29177
3.0,320012


In [33]:
# cc.df = casos_confirmados[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag']]
# cc.df
casos_confirmados.loc[casos_confirmados['hash'].isin(novos_obitos.index.tolist())]

,hash,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,is2,com,evo,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
59763,CARMENLUCIACHAMORRO57410830,115830,3579553.0,PR,410830,410830,CARMEN LUCIA CHAMORRO,F,57,LPGL - Foz do Iguaçu,2020-07-22,2020-07-23,2020-07-12,2.0,2021-06-16,2021-06-17,2020-07-12 00:00:00,2020-07-23,2020-08-25 00:00:00,CARMENLUCIACHAMORRO56410830,CARMENLUCIACHAMORRO58410830,CARMENLUCIACHAMORRO57410830,CARMENLUCIACHAMORRO56410830,CARMENLUCIACHAMORRO58410830,CARMENLUCIACHAMORRO22072020
189965,JANDIRAMASSANEIROKREGENSKI67410180,265510,3289021.0,PR,410180,410180,JANDIRA MASSANEIRO KREGENSKI,F,67,"COVID-19, Biologia Molecular",2020-10-09,2020-10-11,2020-10-03,2.0,2021-05-24,2021-06-17,2020-10-03 00:00:00,2020-10-11,2020-10-14 00:00:00,JANDIRAMASSANEIROKREGENSKI66410180,JANDIRAMASSANEIROKREGENSKI68410180,JANDIRAMASSANEIROKREGENSKI67410180,JANDIRAMASSANEIROKREGENSKI66410180,JANDIRAMASSANEIROKREGENSKI68410180,JANDIRAMASSANEIROKREGENSKI09102020
369230,FILEMONBAEZLESCANO78410830,447079,1766678.0,PR,410830,410830,FILEMON BAEZ LESCANO,M,78,"COVID-19, Biologia Molecular",2020-12-18,2020-12-19,2020-12-15,2.0,2020-12-31,2021-06-17,2020-12-15 00:00:00,2020-12-19,NaN,FILEMONBAEZLESCANO77410830,FILEMONBAEZLESCANO79410830,FILEMONBAEZLESCANO78410830,FILEMONBAEZLESCANO77410830,FILEMONBAEZLESCANO79410830,FILEMONBAEZLESCANO18122020
403517,JACIRADEFATIMASOUZACARVALHO60410550,481418,4044888.0,PR,410550,410550,JACIRA DE FATIMA SOUZA CARVALHO,F,60,"COVID-19, Biologia Molecular",2020-12-27,2020-12-29,2020-12-18,2.0,2021-06-09,2021-06-17,2020-12-18 00:00:00,2020-12-29,2021-01-19 00:00:00,JACIRADEFATIMASOUZACARVALHO59410550,JACIRADEFATIMASOUZACARVALHO61410550,JACIRADEFATIMASOUZACARVALHO60410550,JACIRADEFATIMASOUZACARVALHO59410550,JACIRADEFATIMASOUZACARVALHO61410550,JACIRADEFATIMASOUZACARVALHO27122020
559813,HELENABARBIERIMICHELON85410830,641749,2271327.0,PR,410830,410830,HELENA BARBIERI MICHELON,F,85,Teste Rápido IgM/IgG não discriminatório (Imun...,2021-02-03,2021-02-04,NaT,2.0,2020-10-05,2021-06-17,NaN,2021-02-04,NaN,HELENABARBIERIMICHELON84410830,HELENABARBIERIMICHELON86410830,HELENABARBIERIMICHELON85410830,HELENABARBIERIMICHELON84410830,HELENABARBIERIMICHELON86410830,HELENABARBIERIMICHELON03022021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168983,OEDIODACOSTA59410690,1255919,4265805.0,PR,410690,410690,OEDIO DA COSTA,M,59,1,2021-05-20,2021-06-17,2021-05-20,2.0,2021-06-10,2021-06-17,NaN,NaT,NaN,OEDIODACOSTA58410690,OEDIODACOSTA60410690,OEDIODACOSTA59410690,OEDIODACOSTA58410690,OEDIODACOSTA60410690,NaN
1169199,ADAIRMARTINSNEVES57410690,1256135,4269053.0,PR,410690,410690,ADAIR MARTINS NEVES,M,57,27,2021-06-07,2021-06-17,2021-05-25,2.0,2021-06-14,2021-06-17,NaN,NaT,NaN,ADAIRMARTINSNEVES56410690,ADAIRMARTINSNEVES58410690,ADAIRMARTINSNEVES57410690,ADAIRMARTINSNEVES56410690,ADAIRMARTINSNEVES58410690,NaN
1169273,ANATALICIOALVESDOPRADO86410690,1256209,4270330.0,PR,410690,410690,ANATALICIO ALVES DO PRADO,M,86,27,2021-06-09,2021-06-17,2021-06-09,2.0,2021-06-14,2021-06-17,NaN,NaT,NaN,ANATALICIOALVESDOPRADO85410690,ANATALICIOALVESDOPRADO87410690,ANATALICIOALVESDOPRADO86410690,ANATALICIOALVESDOPRADO85410690,ANATALICIOALVESDOPRADO87410690,NaN
1170438,ONELSOWOITCH79412550,1257374,4285826.0,PR,412550,412550,ONELSO WOITCH,M,79,167,2021-06-04,2021-06-17,2021-06-04,2.0,2021-06-17,2021-06-17,NaN,NaT,NaN,ONELSOWOITCH78412550,ONELSOWOITCH80412550,ONELSOWOITCH79412550,ONELSOWOITCH78412550,ONELSOWOITCH80412550,NaN


In [34]:
cc.df = casos_confirmados[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag']]
cc.save(f"cc_{today.strftime('%d_%m_%Y')}", replace=True)
cc.df.tail()

Load C:\sesa\dev\bulletin\database\casos_confirmados\cc_17_06_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:10


,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
1170627,1257563,4288311.0,PR,412820,412820,MARIA GABRIELA DA SILVA,F,84,27,2021-05-28,2021-06-17,2021-05-28,3.0,NaT,NaT,MARIAGABRIELADASILVA84412820,MARIAGABRIELADASILVA83412820,MARIAGABRIELADASILVA85412820,MARIAGABRIELADASILVA84412820,MARIAGABRIELADASILVA83412820,MARIAGABRIELADASILVA85412820,NaN
1170628,1257564,4288326.0,PR,410840,410840,SILVANA DETUMIM,F,37,167,2021-06-16,2021-06-17,NaT,3.0,NaT,NaT,SILVANADETUMIM37410840,SILVANADETUMIM36410840,SILVANADETUMIM38410840,SILVANADETUMIM37410840,SILVANADETUMIM36410840,SILVANADETUMIM38410840,NaN
1170629,1257565,4288327.0,PR,410490,410490,JOSENALDO APARECIDO DOS SANTOS,M,35,167,2021-06-17,2021-06-17,2021-06-15,3.0,NaT,NaT,JOSENALDOAPARECIDODOSSANTOS35410490,JOSENALDOAPARECIDODOSSANTOS34410490,JOSENALDOAPARECIDODOSSANTOS36410490,JOSENALDOAPARECIDODOSSANTOS35410490,JOSENALDOAPARECIDODOSSANTOS34410490,JOSENALDOAPARECIDODOSSANTOS36410490,NaN
1170630,1257566,4288361.0,PR,411930,411930,EDISON JOSE FONSECA,M,32,167,2021-06-17,2021-06-17,2021-05-15,1.0,2021-05-15,NaT,EDISONJOSEFONSECA32411930,EDISONJOSEFONSECA31411930,EDISONJOSEFONSECA33411930,EDISONJOSEFONSECA32411930,EDISONJOSEFONSECA31411930,EDISONJOSEFONSECA33411930,NaN
1170631,1257567,4288363.0,PR,411990,411990,KAIQUE DE ALMEIDA TEIXEIRA,M,24,27,2021-06-04,2021-06-17,2021-06-02,3.0,NaT,NaT,KAIQUEDEALMEIDATEIXEIRA24411990,KAIQUEDEALMEIDATEIXEIRA23411990,KAIQUEDEALMEIDATEIXEIRA25411990,KAIQUEDEALMEIDATEIXEIRA24411990,KAIQUEDEALMEIDATEIXEIRA23411990,KAIQUEDEALMEIDATEIXEIRA25411990,NaN


In [35]:
casos_confirmados = cc.df
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
0,8838,-1.0,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaT,1.0,2020-04-03,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
1,10932,-1.0,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07,1.0,NaT,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020
2,10933,-1.0,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08,1.0,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
3,11470,-1.0,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11,1.0,NaT,NaT,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE16032020
4,11492,-1.0,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15,1.0,NaT,NaT,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN18032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170627,1257563,4288311.0,PR,412820,412820,MARIA GABRIELA DA SILVA,F,84,27,2021-05-28,2021-06-17,2021-05-28,3.0,NaT,NaT,MARIAGABRIELADASILVA84412820,MARIAGABRIELADASILVA83412820,MARIAGABRIELADASILVA85412820,MARIAGABRIELADASILVA84412820,MARIAGABRIELADASILVA83412820,MARIAGABRIELADASILVA85412820,NaN
1170628,1257564,4288326.0,PR,410840,410840,SILVANA DETUMIM,F,37,167,2021-06-16,2021-06-17,NaT,3.0,NaT,NaT,SILVANADETUMIM37410840,SILVANADETUMIM36410840,SILVANADETUMIM38410840,SILVANADETUMIM37410840,SILVANADETUMIM36410840,SILVANADETUMIM38410840,NaN
1170629,1257565,4288327.0,PR,410490,410490,JOSENALDO APARECIDO DOS SANTOS,M,35,167,2021-06-17,2021-06-17,2021-06-15,3.0,NaT,NaT,JOSENALDOAPARECIDODOSSANTOS35410490,JOSENALDOAPARECIDODOSSANTOS34410490,JOSENALDOAPARECIDODOSSANTOS36410490,JOSENALDOAPARECIDODOSSANTOS35410490,JOSENALDOAPARECIDODOSSANTOS34410490,JOSENALDOAPARECIDODOSSANTOS36410490,NaN
1170630,1257566,4288361.0,PR,411930,411930,EDISON JOSE FONSECA,M,32,167,2021-06-17,2021-06-17,2021-05-15,1.0,2021-05-15,NaT,EDISONJOSEFONSECA32411930,EDISONJOSEFONSECA31411930,EDISONJOSEFONSECA33411930,EDISONJOSEFONSECA32411930,EDISONJOSEFONSECA31411930,EDISONJOSEFONSECA33411930,NaN


In [36]:
municipios = Municipios()

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casos_confirmados = pd.merge(casos_confirmados.rename(columns={'ibge_resid':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_resid'})

casos_confirmados['municipio_pr'] = casos_confirmados['municipio']
casos_confirmados.loc[casos_confirmados['uf_resid']!='PR','municipio_pr'] = 'Fora'

casos_confirmados.loc[casos_confirmados['uf_resid']!='PR','ibge7'] = 9999999

casos_confirmados['uf_pr'] = 'PR'
casos_confirmados.loc[casos_confirmados['uf_resid']!='PR','uf_pr'] = 'Fora'

In [37]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casos_confirmados.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

,PR,Fora,GERAL
CASOS,1.164.281,6.351,1.170.632
OBITOS,29.013,164,29.177
RECUPERADOS,819.092,2.351,821.443
ATIVOS,316.176,3.836,320.012


In [38]:
total_macros = casos_confirmados.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
Leste,539.304,14.665,360.217,164.422
Oeste,238.636,4.891,206.295,27.450
Noroeste,189.648,4.166,142.588,42.894
Norte,196.693,5.291,109.992,81.410
Fora,6.351,164,2.351,3.836


In [39]:
total_regionais = casos_confirmados.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
rs,regional,,,,
1,Paranaguá,36.242,916,19.027,16.299
2,Metropolitana,342.596,10.110,238.568,93.918
3,Ponta Grossa,68.930,1.679,45.628,21.623
4,Irati,16.848,353,12.647,3.848
5,Guarapuava,37.532,807,22.375,14.350
6,União da Vitória,12.631,257,9.126,3.248
7,Pato Branco,31.449,737,27.121,3.591
8,Francisco Beltrão,42.476,728,36.595,5.153
9,Foz do Iguaçu,56.973,1.289,44.865,10.819


In [40]:
total_municipios = casos_confirmados.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

evolucao                                         CASOS  OBITOS  RECUPERADOS  \
macro rs regional  ibge7   municipio_pr                                       
Leste 1  Paranaguá 4101200 Antonina              2.413      61        2.175   
                   4109500 Guaraqueçaba            648      18          548   
                   4109609 Guaratuba             4.383     160        3.950   
                   4115705 Matinhos              2.988     110        2.593   
                   4116208 Morretes              2.046      50        1.632   
...                                                ...     ...          ...   
Norte 22 Ivaiporã  4122172 Rio Branco do Ivaí      283      10          263   
                   4122651 Rosário do Ivaí         381       2          376   
                   4123857 Santa Maria do Oeste    694      23          619   
                   4125001 São João do Ivaí        984      20          856   
Fora  99 Fora      9999999 Fora                  6.351     164        2.351   

evolucao                                         ATIVOS  
macro rs regional  ibge7   municipio_pr                  
Leste 1  Paranaguá 4101200 Antonina                 177  
                   4109500 Guaraqueçaba              82  
                   4109609 Guaratuba                273  
                   4115705 Matinhos                 285  
                   4116208 Morretes                 364  
...                                                 ...  
Norte 22 Ivaiporã  4122172 Rio Branco do Ivaí        10  
                   4122651 Rosário do Ivaí            3  
                   4123857 Santa Maria do Oeste      52  
                   4125001 São João do Ivaí         108  
Fora  99 Fora      9999999 Fora                   3.836  

[400 rows x 4 columns]

In [41]:
ministerio_cols = ['rs','municipio_pr','CASOS','OBITOS']
total_municipios.reset_index()[ministerio_cols].sort_values(['rs','municipio_pr']).to_excel(join(default_output,'ministerio.xlsx'),index=False)

In [42]:
casos_confirmados['regional'] = casos_confirmados['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casos_confirmados['regional']
casos_confirmados = casos_confirmados[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

In [43]:
obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']=='OBITO']

In [44]:
writer = pd.ExcelWriter(join(default_output, f"casos_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [45]:
writer = pd.ExcelWriter(join(default_output, f"obitos_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = obitos_confirmados.loc[obitos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [46]:
# casos_confirmados['rs'].unique()

In [47]:
writer = pd.ExcelWriter(join(default_output,f"casos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(casos_confirmados['regional'].unique()):
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional['macro'] = casos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [48]:
writer = pd.ExcelWriter(join(default_output,f"obitos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(obitos_confirmados['regional'].unique()):
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional['macro'] = obitos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()

In [49]:
# REMOVE ARQUIVOS DE ONTEM
try:
    remove(join(default_output, f"casos_{ontem.strftime('%d_%m_%Y')}.xlsx"))
    remove(join(default_output, f"obitos_{ontem.strftime('%d_%m_%Y')}.xlsx"))
    remove(join(default_output, f"casos_regionais_{ontem.strftime('%d_%m_%Y')}.xlsx"))
    remove(join(default_output, f"obitos_regionais_{ontem.strftime('%d_%m_%Y')}.xlsx"))

    remove(join(default_output, f"casos_{ontem.strftime('%d_%m_%Y')}.zip"))
    remove(join(default_output, f"obitos_{ontem.strftime('%d_%m_%Y')}.zip"))
    remove(join(default_output, f"casos_regionais_{ontem.strftime('%d_%m_%Y')}.zip"))
    remove(join(default_output, f"obitos_regionais_{ontem.strftime('%d_%m_%Y')}.zip"))
except:
    pass

# GERA ARQUIVOS COMPACTADOS DE HOJE

paths = [join(default_output, f"casos_{today.strftime('%d_%m_%Y')}.xlsx")]
paths.append(join(default_output, f"obitos_{today.strftime('%d_%m_%Y')}.xlsx"))
paths.append(join(default_output, f"casos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"))
paths.append(join(default_output, f"obitos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"))

for path in paths:
    pz.compress(path, None, join(default_output, f"{basename(path)}.zip".replace(".xlsx", "")), "Jt\j+ZXVOAbSw*i&J)ZePVGiQgU4wc", 9)


# REMOVE ARQUIVOS XLSX DE HOJE
remove(join(default_output, f"casos_{today.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output, f"obitos_{today.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output, f"casos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output, f"obitos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"))

<ipython-input-49-c64eec2abfb0>:23: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  pz.compress(path, None, join(default_output, f"{basename(path)}.zip".replace(".xlsx", "")), "Jt\j+ZXVOAbSw*i&J)ZePVGiQgU4wc", 9)
